В этом задании мы должны научить машину понимать тексты. В файле recs.txt находятся отзывы о разных фильмах. Наша модель машинного обучения должна проанализировать отзыв и ответить на простой вопрос: понравился ли пользователю просмотренный фильм.

Подключите необходимые для работы пакеты: pandas as pd, numpy as np, nltk, re, все функции из sklearn

Увеличьте ширину выводимой информации в DataFrame (на это понадобится для показа текстов отзывав):
pd.set_option('max_colwidth',600)

Прочтите данные из файла recs.txt, в качестве разделителя (параметр sep) там выступает строка "\\-\\-\\|\\|\\|\\-\\-", кодировка (параметр encoding) - 'utf-8'.

Выведите три первых строки таблицы. Какое отношение (Sentiment) у авторов отзывов к просмотренным фильмам?

Чтобы преобразовать тексты в наборы признаков методом bag-of-words мы должны сначала профильтровать текст, оставив только значащие слова. Скачайте из корпуса набор незначащих слов для фильтрации командой nltk.download()  (в появившемся окне вкладка Corpora, пункт stopwords). 

Импортируйте список незначащих слов командами 
```python
from nltk.corpus import stopwords
stopwords = set(stopwords.words("russian"))
```
и выведите его на экран.

Определите функцию фильтрации текста:
```python
def review_to_words( review_text ):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", review_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    # 4. Формируем текст, объединяя слова через пробел
    return( " ".join( meaningful_words ))
```

Скопируйте базу данных отзывов с помощью метода copy() в новую переменную и примените к отзывам функцию фильтрации: .apply(lambda s: review_to_words(s)).

Выведите на экран первые три строчки преобразованной базы данных

Запустите преобразователь текстового признака в «bag of words» с помощью класса CountVectorizer библиотеки sklearn.feature_extraction.text. 

В конструкторе класса укажите параметры: analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 50000, binary = True

Вызовите метод обучения (построения словаря) и преобразования текстов: fit_transform. Результат присвойте новой переменной bagOfWords.

Сохраните порядок слов в словаре (метод .get_feature_names()) в переменную vocabulary.

Теперь в наших матрицах объектов-признаков 50 тысяч столбцов и в сумме 100 тысяч строк. Если 1 элемент занимает 1 байт, то суммарный объем будет 5 Гб. Проверьте по диспетчеру задач (системный монитор в Linux), сколько занимает памяти ваша программа и напишите в комментариях. Все ли согласуется?

Разбейте выборку bagOfWords на две части: обучающую (строки до 50000) и проверочную (строки >= 50000). Назовите их bagOfWordsTrain и bagOfWordsTest

Сконструируйте машину опорных векторов svm.LinearSVC с параметрами C=1, fit_intercept=False и присвойте ее переменной SVM.

Натренируйте ее на обучающей выборке и с помощью метода .score(x,y) вычислите процент правильных предсказаний для тестовой выборки.

Подберите важность суммы нарушений C, чтобы SVM работала лучше всего

Выведите на экран 20 слов, которые больше всего голосуют за позитивность отзыва и 20 — за негативность вместе с соответствующими им коэффициентами натренированной линейной функции:
```python
ind = np.argsort(SVM.coef_[0])
for i in range(20):
    print(SVM.coef_[0][ind[i]], vocabulary[ind[i]])
print("\n")
for i in range(20):
    print(SVM.coef_[0][ind[-i-1]], vocabulary[ind[-i-1]]) 
```

Какой коэффициент соответствует словам: 'критика', 'странно', 'понимаю', 'деньги', 'даже' и еще 5 ваших слов по выбору. За что они голосуют: позитивность или негативность отзыва?

Вычислите выступы, найдите и выведете на экран отзывы с минимальным и максимальным выступом, величину выступа, величину предсказания и сентиментность отзыва. Объясните в комментариях, почему найденный отзыв имеет такой маленький выступ, а другой найденный — такой большой:
```python
prediction = SVM.decision_function(bagOfWordsTest)
margins = np.multiply(prediction, data.loc[50000:]['Sentiment'].as_matrix())
ind = np.argsort(margins)
i = ind[-1] + 50000
print('Величина выступа = ', margins[ind[-1]])
print('Величина предсказания = ', prediction[ind[-1]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])
print("\n")
i = ind[0] + 50000
print('Величина выступа = ', margins[ind[0]])
print('Величина предсказания = ', prediction[ind[0]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])
```

Для объяснения отступов вам возможно пригодится код, который печатает коэффициенты w<sub>i</sub> линейной функции, значения соответствующих признаков и соответствующие слова. И все это делается в порядке убывания модуля произведения w<sub>i</sub>x<sub>i</sub>:
```python
k = ind[0]
ind2 = np.argsort(np.abs(np.multiply(SVM.coef_[0], bagOfWordsTest[k].todense()))).reshape(-1,1)
s = 0
for i in range((bagOfWordsTest[k]!=0).sum()):
    ii = ind2[-i-1].item()
    print('wi='+str(SVM.coef_[0][ii]), 'xi='+str(bagOfWordsTest[k,ii]),  vocabulary[ii])
    s += SVM.coef_[0][ii]*bagOfWordsTest[k,ii]
print('Проверка:',s) 
```

Нормализуйте признаки перед разбиением выборки на Train и Test так, чтобы вместо 0/1 стояли частоты употребления слов, деленные на длину отзыва. Подберите наилучшую константу C и опишите, как изменятся отзывы с минимальным и максимальным выступом? Что лучше: не нормализованные или нормализованные признаки? Почему?

Нужно установить binary = False в CountVectorizer и добавить после fit_transform
```python
bagOfWords = preprocessing.normalize(bagOfWords, norm='l1')
```